In [40]:
trait Ord {
    // this cmp that < 0 iff this < that
    // this cmp that > 0 iff this > that
    // this cmp that == 0 iff this == that
    def cmp(that: Ord): Int
    def ===(that: Ord): Boolean = (this.cmp(that)) == 0
    def < (that: Ord): Boolean = (this cmp that) < 0
    def > (that: Ord): Boolean = (this cmp that) > 0
    def <= (that: Ord): Boolean = (this cmp that) <= 0
    def >= (that: Ord): Boolean = (this cmp that) >= 0
}
def max3(a: Ord, b: Ord, c: Ord) : Ord =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [41]:
case class OInt(val value: Int) extends Ord {
    def cmp(that: Ord): Int = {
        that match {
            case OInt(v) => value - v
        }
    }
}
case class OString(val value: String) extends Ord {
    def cmp(that: Ord): Int = {
        that match {
            case OString(v) => value.compare(v)
        }
    }
}

max3(OInt(3), OInt(2), OInt(10))
max3(OString("abc"), OString("aaa"), OString("a"))

defined class OInt
defined class OString
res40_2: Ord = OInt(10)
res40_3: Ord = OString("abc")

### Make `Ord` to `Ord[A]`

In [45]:
trait Ord[A] {
    def cmp(that: Ord[A]): Int
    def ===(that: Ord[A]): Boolean = (this.cmp(that)) == 0
    def < (that: Ord[A]): Boolean = (this cmp that) < 0
    def > (that: Ord[A]): Boolean = (this cmp that) > 0
    def <= (that: Ord[A]): Boolean = (this cmp that) <= 0
    def >= (that: Ord[A]): Boolean = (this cmp that) >= 0
    def value: A
}

def max3[A](a: Ord[A], b: Ord[A], c: Ord[A]): Ord[A] =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [46]:
case class OInt(val value: Int) extends Ord[Int] {
    def cmp(that: Ord[Int]): Int = value - that.value
}
case class OString(val value: String) extends Ord[String] {
    def cmp(that: Ord[String]): Int = value.compare(that.value)
}
max3(OInt(3), OInt(2), OInt(10)).value
max3(OString("abc"), OString("aaa"), OString("a")).value

defined class OInt
defined class OString
res45_2: Int = 10
res45_3: String = "abc"

In [47]:
case class OInt2(val value: Int) extends Ord[Int] {
    def cmp(that: Ord[Int]): Int = that.value - value
}
max3(OInt(1), OInt2(2), OInt(10)).value

defined class OInt2
res46_1: Int = 2

### Not to use `def value`

In [51]:
trait Ord[A] {
    def cmp(that: A): Int
    def ===(that: A): Boolean = (this.cmp(that)) == 0
    def < (that: A): Boolean = (this cmp that) < 0
    def > (that: A): Boolean = (this cmp that) > 0
    def <= (that: A): Boolean = (this cmp that) <= 0
    def >= (that: A): Boolean = (this cmp that) >= 0
}

def max3[A <: Ord[A]](a: A, b: A, c: A): A =
    if (a <= b) { if (b <= c) c else b }
    else        { if (a <= c) c else a }

defined trait Ord
defined function max3

In [56]:
case class OInt(val value: Int) extends Ord[OInt] {
    def cmp(that: OInt): Int = value - that.value
}
case class OString(val value: String) extends Ord[OString] {
    def cmp(that: OString): Int = value.compare(that.value)
}
max3(OInt(3), OInt(2), OInt(10))
max3(OString("abc"), OString("aaa"), OString("a"))

defined class OInt
defined class OString
res55_2: OInt = OInt(10)
res55_3: OString = OString("abc")

In [58]:
case class OInt2(val value: Int) extends Ord[OInt] {
    def cmp(that: OInt): Int = that.value - value
}
max3(OInt(1), OInt2(2), OInt(10))

cmd58.sc:4: inferred type arguments [Product with cmd58.this.cmd50.Ord[cmd58.this.cmd55.OInt] with java.io.Serializable] do not conform to method max3's type parameter bounds [A <: cmd58.this.cmd50.Ord[A]]
val res58_1 = max3(OInt(1), OInt2(2), OInt(10))
              ^cmd58.sc:4: type mismatch;
 found   : cmd58.this.cmd55.OInt
 required: A
val res58_1 = max3(OInt(1), OInt2(2), OInt(10))
                       ^cmd58.sc:4: type mismatch;
 found   : Helper.this.OInt2
 required: A
val res58_1 = max3(OInt(1), OInt2(2), OInt(10))
                                 ^cmd58.sc:4: type mismatch;
 found   : cmd58.this.cmd55.OInt
 required: A
val res58_1 = max3(OInt(1), OInt2(2), OInt(10))
                                          ^Compilation Failed

: 

In [59]:
class OInt2(val value2: Int) extends OInt(value2) {
    override def cmp(that: OInt): Int = that.value - value2
}
max3(OInt(1), new OInt2(2), OInt(10))

defined class OInt2
res58_1: OInt = OInt(2)